In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("ordinal")
install.packages("dplyr")
library(ordinal)
library(dplyr)

# Data and Setup

Link to the GSS 2006 Codebook - https://www.thearda.com/Archive/Files/Codebooks/GSS2006_CB.asp

Link to the WVS Codebook- https://www.thearda.com/Archive/Files/Codebooks/WVS2010_CB.asp

In [ ]:
%%R
gss_2006 = read.csv('/gdrive/My Drive/Data Analysis GR5015/GSS_2006_enc.csv')

### Question 1
***1.Run a multiple ordinal logistic regression. Tell me what you expect to find and then interpret your results to tell me what you did infact find.***

#### **Observations & Interpretations**
- From the coefficients - A unit increase in education on the scale increases the likelihood of Work being the top satisfaction by 0.049 units, on an average, on the log-odds scale, holding all other variables constant. This interpretation is statistically significant given the p-value is 0.003 (which is much less than or equal to 0.05 i.e a rule of thumb)

- From the coefficients - A unit increase in sex, that is going from men to women, decreases the likelihood of Work being the top satisfaction by 0.19 units, on an average, on the log-odds scale, holding all other variables constant. This interpretation is statistically significant given the p-value is 0.036 (which is less than or equal to 0.05 i.e a rule of thumb)


#### **My intuition and if it worked out**

My intuition was that more qualified people draw a lot of inspiration and identity from their qualification and application of their qualification and hence would find higher utility/satisfaction from their work/career.

My intuition with age was that women at certain point in their lives might replace work as the top satisfaction with probably a happy/healthy/stable household or family and this maybe a phase in the middle years of their career, But may find work to agin become top source of satissfaction.

Both my intuitions are well justified by the regression outcome. Although i am curious to further see this with age and sex alone and probably use panel data to see if it changes for same set of people with increaseing age as they go through phases of life



In [ ]:
%%R
gss_2006 %>%
  group_by(educ) %>%
  summarise(n= n())

# A tibble: 22 x 2
    educ     n
   <int> <int>
 1     0    22
 2     1     4
 3     2    28
 4     3    13
 5     4    11
 6     5    23
 7     6    69
 8     7    32
 9     8    85
10     9   127
# … with 12 more rows


In [ ]:
#Varaiable 'educ' is on a scale of 1-20 with incresing value means more education qualification
#Varaiable 'sex' is assigned as male=1 and female=2


# Re-coding the "My main satisfaction in life comes from work." variable to denote a higher value means higher their agreement to the statement 
%%R
gss_2006$wktopsat = 5-gss_2006$wktopsat

In [ ]:
%%R
# Running a Ordinal Logit regression
fm1 <- clm(as.factor(wktopsat) ~ educ + sex, data=gss_2006)
summary(fm1)

formula: as.factor(wktopsat) ~ educ + sex
data:    gss_2006

 link  threshold nobs logLik   AIC     niter max.grad cond.H 
 logit flexible  1707 -2018.98 4047.96 5(0)  9.50e-08 2.5e+04

Coefficients:
     Estimate Std. Error z value Pr(>|z|)   
educ  0.04985    0.01716   2.905  0.00367 **
sex  -0.18996    0.09062  -2.096  0.03606 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
    Estimate Std. Error z value
1|2  -0.7987     0.2813  -2.840
2|3   1.3555     0.2830   4.789
3|4   3.2509     0.2979  10.911
(2803 observations deleted due to missingness)


### Question 2
***2. Test the proportional odds assumption of the ordinal logistic regression. Tell me what the test found? Is the assumption satisfied or not?***

#### **Observations & Interpretations**

- From the coefficients - A unit increase in sex, that is going from men to women, decreases the likelihood of Work being the top satisfaction in life by 0.19 units, on an average, on the log-odds scale, holding all other variables constant. This interpretation is statistically significant given the p-value is 0.036 (which is less than or equal to 0.05 i.e a rule of thumb)

- The coefficients we see of this model we fit can be interpreted in the following ways- For 1|2 the Likelihoold of posseing 1 value on education over 2,3 or 4 is 0.94 or 94% and cumulative goes from here.  For 2|3 the Likelihoold of posseing a value of 1 or 2  on education over 3 or 4 is 0.9544 i.e 95.44%


The Ordinal Logit assumes that the odds-ration(the chance of being in one category over others hods when compared between all pairs of possibilities)  meaning the results will be a set of parallel lines, this is what we test using the Brandt test which is performed at the end.


To qualify the Brandts test we look the chi-sqquared test output and use it to reject the Null hypothesis (That lines are parallel). In our output the Chi-squared test value is 0.1 which is higher than (0.05) hence we reject the null hypothesis and say that the proportional odds assumption holds up in the ordinal logit regression 


In [ ]:
#To test the proportional odds we nominalize the Education variable and run the Ordinal Logit Regression
%%R
fm2 <- clm(as.factor(wktopsat) ~ sex, nominal=~educ, data=gss_2006) # nominalizing the Education vriable
summary(fm2)

formula: as.factor(wktopsat) ~ sex
nominal: ~educ
data:    gss_2006

 link  threshold nobs logLik   AIC     niter max.grad cond.H 
 logit flexible  1707 -2017.25 4048.51 6(0)  2.63e-13 3.7e+04

Coefficients:
    Estimate Std. Error z value Pr(>|z|)  
sex -0.18972    0.09066  -2.093   0.0364 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
                 Estimate Std. Error z value
1|2.(Intercept) -0.634199   0.341083  -1.859
2|3.(Intercept)  1.307866   0.321965   4.062
3|4.(Intercept)  2.404916   0.519787   4.627
1|2.educ        -0.061689   0.022149  -2.785
2|3.educ        -0.046638   0.020285  -2.299
3|4.educ         0.009464   0.035119   0.269
(2803 observations deleted due to missingness)


In [ ]:
%%R
exp(coef(fm2))

1|2.(Intercept) 2|3.(Intercept) 3|4.(Intercept)        1|2.educ        2|3.educ 
      0.5303600       3.6982735      11.0774975       0.9401754       0.9544327 
       3|4.educ             sex 
      1.0095088       0.8271871 


In [ ]:
# We perform the Brandt test here to compare and see if the above two regressions have the  parallell slopes among them

%%R
anova(fm1,fm2) 

Likelihood ratio tests of cumulative link models:
 
    formula:                         nominal: link: threshold:
fm1 as.factor(wktopsat) ~ educ + sex ~1       logit flexible  
fm2 as.factor(wktopsat) ~ sex        ~educ    logit flexible  

    no.par    AIC  logLik LR.stat df Pr(>Chisq)
fm1      5 4048.0 -2019.0                      
fm2      7 4048.5 -2017.3  3.4594  2     0.1773
